# External data

We are adding more information to the original CSV file regularite-mensuelle-tgv-aqst.csv in order to have more explanations, and better results on our prediction.

## Monthly regularity (original dataset)

When merging with the other dataset found in the SNCF website, we found that some of the station's name in this dataset are different from the other ones. Since the number of differences are acceptable, we decide to correct manually the names to match the other datasets. 

In [22]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [23]:
df = pd.read_csv('regularite-mensuelle-tgv-aqst.csv',sep=';')
df.replace({'BORDEAUX ST JEAN':'BORDEAUX SAINT-JEAN', 'LA ROCHELLE VILLE':'LA ROCHELLE', 'ST PIERRE DES CORPS': 'SAINT-PIERRE-DES-CORPS',
       'LILLE':'LILLE EUROPE', 'PARIS VAUGIRARD': 'PARIS MONTPARNASSE', 'CHAMBERY CHALLES LES EAUX':'CHAMBÉRY - CHALLES-LES-EAUX',
       'MONTPELLIER':'MONTPELLIER SAINT-ROCH', 'MULHOUSE VILLE':'MULHOUSE', 'NICE VILLE':'NICE', 'PARIS LYON':'PARIS GARE DE LYON',
       'PARIS NORD':'PARIS GARE DU NORD', 'BELLEGARDE (AIN)': 'BELLEGARDE',
       'MACON LOCHE':'MÂCON LOCHÉ TGV', 'MARNE LA VALLEE': 'MARNE-LA-VALLÉE CHESSY', 'ANGERS SAINT LAUD': 'ANGERS SAINT-LAUD',
       'BESANCON FRANCHE COMTE TGV': 'BESANÇON FRANCHE-COMTÉ TGV', 'NIMES':'NÎMES',
       'SAINT ETIENNE CHATEAUCREUX':'SAINT-ÉTIENNE CHÂTEAUCREUX',
       'ANGOULEME': 'ANGOULÊME', 'MARSEILLE ST CHARLES': 'MARSEILLE SAINT-CHARLES', 'VALENCE ALIXAN TGV':'VALENCE TGV RHÔNES-ALPES SUD',
       'ST MALO': 'SAINT-MALO', 'AIX EN PROVENCE TGV': 'AIX-EN-PROVENCE TGV', 'DIJON VILLE': 'DIJON',
       'LE CREUSOT MONTCEAU MONTCHANIN': 'LE CREUSOT - MONTCEAU-LES-MINES - MONTCHANIN TGV'}, inplace=True)

## Found Objects
Due to the vigipirate plan, abandoned luggage (bag and suitcase) can cause complications in the station: intervention by specialized teams, interrupted traffic, blocked access to the platform, etc.
We therefore want to add the number of bags lost in each station, and see if it can explain delays, particularly on the external causes of train delays.

In [24]:

freq = pd.read_csv('external_data/frequentation-gares.csv',sep=';')


incidents = pd.read_csv('external_data/incidents-securite.csv', sep=';')
liaison = pd.read_csv('external_data/lignes-equipees-de-liaison-radio-sol-train.csv', sep=';')
cantonnement = pd.read_csv('external_data/mode-de-cantonnement-des-lignes.csv', sep=';')


In [25]:
objet = pd.read_csv('external_data/objets-trouves-restitution.csv', sep=';')
objet.head()

,Date,Date et heure de restitution,Gare,Code UIC,Nature d'objets,Type d'objets,Type d'enregistrement
0,2018-01-05T07:54:17+01:00,2018-01-08T15:52:30+01:00,Dijon,87713040.0,"Valise, sac sur roulettes","Bagagerie: sacs, valises, cartables",Objet trouvé
1,2018-01-05T08:19:10+01:00,NaN,Châtellerault,87575142.0,"Parapluie télescopique, parapluie de poche",Parapluies,Objet trouvé
2,2018-01-05T08:21:06+01:00,NaN,Creil,87276006.0,"Attaché-case, serviette","Bagagerie: sacs, valises, cartables",Objet trouvé
3,2018-01-05T08:24:25+01:00,2018-01-09T09:41:26+01:00,Nantes,87481002.0,Lunettes en étui,Optique,Objet trouvé
4,2018-01-05T08:43:24+01:00,NaN,Châtellerault,87575142.0,"Parapluie-canne, parapluie à bandoulière",Parapluies,Objet trouvé


In [26]:
#only months and years are interesting for us regarding the original dataset that we have
objet['Date'] = objet['Date'].astype(str).str[0:7]
objet['Nature d\'objets'] = objet['Nature d\'objets'].str.lower() 
objet.head()

,Date,Date et heure de restitution,Gare,Code UIC,Nature d'objets,Type d'objets,Type d'enregistrement
0,2018-01,2018-01-08T15:52:30+01:00,Dijon,87713040.0,"valise, sac sur roulettes","Bagagerie: sacs, valises, cartables",Objet trouvé
1,2018-01,NaN,Châtellerault,87575142.0,"parapluie télescopique, parapluie de poche",Parapluies,Objet trouvé
2,2018-01,NaN,Creil,87276006.0,"attaché-case, serviette","Bagagerie: sacs, valises, cartables",Objet trouvé
3,2018-01,2018-01-09T09:41:26+01:00,Nantes,87481002.0,lunettes en étui,Optique,Objet trouvé
4,2018-01,NaN,Châtellerault,87575142.0,"parapluie-canne, parapluie à bandoulière",Parapluies,Objet trouvé


In [27]:
objet['Type d\'objets'].unique()

array(['Bagagerie: sacs, valises, cartables', 'Parapluies', 'Optique',
       'Porte-monnaie / portefeuille, argent, titres',
       'Appareils électroniques, informatiques, appareils photo',
       'Vêtements, chaussures', 'Divers',
       "Pièces d'identités et papiers personnels",
       "Articles d'enfants, de puériculture",
       'Vélos, trottinettes, accessoires 2 roues',
       'Livres, articles de papéterie',
       'Articles de sport, loisirs, camping', 'Bijoux, montres',
       'Clés, porte-clés, badge magnétique', 'Articles médicaux',
       'Instruments de musique'], dtype=object)

In [28]:
# Vigipirate plan is activated for the bag category if no owner is found 
objet = objet[objet['Type d\'objets'] == 'Bagagerie: sacs, valises, cartables']
objet.head()

,Date,Date et heure de restitution,Gare,Code UIC,Nature d'objets,Type d'objets,Type d'enregistrement
0,2018-01,2018-01-08T15:52:30+01:00,Dijon,87713040.0,"valise, sac sur roulettes","Bagagerie: sacs, valises, cartables",Objet trouvé
2,2018-01,NaN,Creil,87276006.0,"attaché-case, serviette","Bagagerie: sacs, valises, cartables",Objet trouvé
6,2018-01,NaN,Toulouse Matabiau,87611004.0,sac à dos,"Bagagerie: sacs, valises, cartables",Objet trouvé
7,2018-01,NaN,Toulouse Matabiau,87611004.0,sac à dos,"Bagagerie: sacs, valises, cartables",Objet trouvé
10,2018-01,NaN,Paris Montparnasse,87391003.0,"sac d'enseigne (plastique, papier, …)","Bagagerie: sacs, valises, cartables",Objet trouvé


In [29]:
# Group by to know the number of lost bags every month in each station
objet = objet.groupby(['Date', 'Gare']).count()['Code UIC'].reset_index()
objet.rename(columns={'Code UIC': 'Nombre objets perdus'}, inplace=True)
objet.head(10)

,Date,Gare,Nombre objets perdus
0,2013-05,Paris Est,8
1,2013-05,Paris Saint-Lazare,17
2,2013-05,Saint-Étienne Châteaucreux,8
3,2013-05,Strasbourg,18
4,2013-06,Colmar,2
5,2013-06,Metz Ville,1
6,2013-06,Nancy,17
7,2013-06,Paris Est,75
8,2013-06,Paris Saint-Lazare,71
9,2013-06,Reims,1


In [30]:
objet.tail(10)

,Date,Gare,Nombre objets perdus
13512,2023-10,Toulouse Matabiau,45
13513,2023-10,Tourcoing,2
13514,2023-10,Tours,25
13515,2023-10,Trouville - Deauville,2
13516,2023-10,Valence,15
13517,2023-10,Valence TGV Rhône-Alpes Sud,3
13518,2023-10,Vannes,1
13519,2023-10,Vierzon,2
13520,2023-10,Évian-les-Bains,1
13521,2023-10,Évreux Normandie,1


In [31]:
objet['Gare']  = objet['Gare'].str.upper() 
df = pd.merge(df, objet, how='left', left_on=['date', 'gare_depart'], right_on=['Date', 'Gare'])
df.drop(columns=['Date', 'Gare'], inplace=True)
df = pd.merge(df, objet, how='left', left_on=['date', 'gare_arrivee'], right_on=['Date', 'Gare'])
df.drop(columns=['Date', 'Gare'], inplace=True)
df.rename(columns={'Nombre objets perdus_x':'nombre_objets_perdus_depart', 'Nombre objets perdus_y': 'nombre_objets_perdus_arrivee'}, inplace=True)
df.fillna({'nombre_objets_perdus_depart':0, 'nombre_objets_perdus_arrivee':0}, inplace=True)
df.head()

,date,service,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,commentaire_annulation,nb_train_depart_retard,retard_moyen_depart,...,nb_train_retard_sup_30,nb_train_retard_sup_60,prct_cause_externe,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs,nombre_objets_perdus_depart,nombre_objets_perdus_arrivee
0,2018-01,National,BORDEAUX SAINT-JEAN,PARIS MONTPARNASSE,141,870,5,NaN,289,11.247809,...,44,8,36.134454,31.092437,10.924370,15.966387,5.042017,0.840336,105.0,157.0
1,2018-01,National,LA ROCHELLE,PARIS MONTPARNASSE,165,222,0,NaN,8,2.875000,...,5,0,15.384615,30.769231,38.461538,11.538462,3.846154,0.000000,22.0,157.0
2,2018-01,National,PARIS MONTPARNASSE,QUIMPER,220,248,1,NaN,37,9.501351,...,17,7,26.923077,38.461538,15.384615,19.230769,0.000000,0.000000,157.0,26.0
3,2018-01,National,PARIS MONTPARNASSE,SAINT-MALO,156,102,0,NaN,12,19.912500,...,6,4,23.076923,46.153846,7.692308,15.384615,7.692308,0.000000,157.0,0.0
4,2018-01,National,PARIS MONTPARNASSE,SAINT-PIERRE-DES-CORPS,61,391,2,NaN,61,7.796995,...,6,0,21.212121,42.424242,9.090909,21.212121,6.060606,0.000000,157.0,4.0


In [32]:
df['nombre_objets_perdus_depart'].unique()

array([105.,  22., 157.,  26.,  77.,   4.,  38.,  96.,  42., 107.,   9.,
        98.,  23.,  19.,  87.,  54.,  52.,  76., 194.,   0.,  34.,  14.,
        64.,   7.,   5., 127.,   6.,  11.,  13.,  16.,  31.,   3.,   2.,
       106.,   8., 140.,  70.,  48.,  90.,  28.,  24., 147.,  18.,  12.,
       129.,  93.,  65.,   1.,  94., 136.,  36.,  53.,  17.,  29.,  68.,
        27., 141., 115.,  59., 132.,  62., 188.,  40.,  79.,  51., 102.,
        86.,  75.,  39.,  10., 111.,  92.,  20., 134.,  58., 158.,  66.,
        71.,  49.,  21., 125.,  60.,  89.,  97., 166.,  45., 148.,  73.,
        15., 139.,  99.,  80.,  46.,  81.,  44., 164., 114., 249.,  47.,
       101.,  37.,  33., 112., 137., 142.,  91.,  78., 145., 165.,  72.,
        32.,  30.,  43., 133., 104., 176., 192., 144., 186., 100.,  88.,
       154.,  85., 113., 173., 108., 123., 103., 181.,  56., 215.,  55.,
        74., 130.,  41., 218.,  50., 109.,  25.,  57., 116.,  67., 117.,
       205., 122., 151.,  84., 187.,  61., 167., 16

## Geographical references

In [33]:
geo = pd.read_csv('external_data/referentiel-gares-voyageurs.csv',sep=';')
geo.head()

,Code plate-forme,Code gare,Code UIC,Date fin validité plateforme,Intitulé plateforme,Code postal,Code Commune,Commune,Code département,Département,...,RG,Intitulé gare,Intitulé fronton de gare,DTG,Région SNCF,Unité gare,UT,Nbre plateformes,TVS,WGS 84
0,00002-1,2,87988709,NaN,Remise à Jorelle,93140.0,10.0,Bondy,93.0,Seine-Saint-Denis,...,GARES B IDF LIGNE T4,Remise à Jorelle,Remise à Jorelle,DGIF,REGION DE PARIS-EST,NaN,BONDY GARE REMISE A JORELLE TRAM TRAIN,1,RJL,"48.89317, 2.487751"
1,00005-1,5,87784892,NaN,Bena Fanes,66760.0,66.0,Enveitg,66.0,Pyrénées-Orientales,...,GARES C LANGUEDOC ROUSSILLON,Bena Fanes,Bena Fanes,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,ENVEITG HALTE DE BENA FANES,1,BFA,"42.4580782, 1.9167264"
2,00006-1,6,87784884,NaN,Ur - Les Escaldes,66760.0,218.0,Ur,66.0,Pyrénées-Orientales,...,GARES C LANGUEDOC ROUSSILLON,Ur - Les Escaldes,Ur les Escaldes,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,UR LES ESCALDES GARE,1,URL,"42.457481, 1.9404821"
3,00010-1,10,87784843,NaN,Err,66800.0,67.0,Err,66.0,Pyrénées-Orientales,...,GARES C LANGUEDOC ROUSSILLON,Err,Err,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,ERR GARE,1,ERR,"42.442777, 2.0290375"
4,00024-1,24,87784702,NaN,Serdinya,66360.0,193.0,Serdinya,66.0,Pyrénées-Orientales,...,GARES C LANGUEDOC ROUSSILLON,Serdinya,Serdinya,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,SERDINYA GARE,1,ZDI,"42.567023, 2.3230869"


In [34]:
geo['Intitulé gare']  = geo['Intitulé gare'].str.upper() 
geo.dropna(subset=['Longitude','Latitude'], inplace=True)
geo.drop_duplicates(subset=['Intitulé gare'])
geo = geo[['Intitulé gare', 'Longitude', 'Latitude']]

In [35]:
geo.head()

,Intitulé gare,Longitude,Latitude
0,REMISE À JORELLE,2.487751,48.893170
1,BENA FANES,1.916726,42.458078
2,UR - LES ESCALDES,1.940482,42.457481
3,ERR,2.029037,42.442777
4,SERDINYA,2.323087,42.567023


In [36]:
df.head()

,date,service,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,commentaire_annulation,nb_train_depart_retard,retard_moyen_depart,...,nb_train_retard_sup_30,nb_train_retard_sup_60,prct_cause_externe,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs,nombre_objets_perdus_depart,nombre_objets_perdus_arrivee
0,2018-01,National,BORDEAUX SAINT-JEAN,PARIS MONTPARNASSE,141,870,5,NaN,289,11.247809,...,44,8,36.134454,31.092437,10.924370,15.966387,5.042017,0.840336,105.0,157.0
1,2018-01,National,LA ROCHELLE,PARIS MONTPARNASSE,165,222,0,NaN,8,2.875000,...,5,0,15.384615,30.769231,38.461538,11.538462,3.846154,0.000000,22.0,157.0
2,2018-01,National,PARIS MONTPARNASSE,QUIMPER,220,248,1,NaN,37,9.501351,...,17,7,26.923077,38.461538,15.384615,19.230769,0.000000,0.000000,157.0,26.0
3,2018-01,National,PARIS MONTPARNASSE,SAINT-MALO,156,102,0,NaN,12,19.912500,...,6,4,23.076923,46.153846,7.692308,15.384615,7.692308,0.000000,157.0,0.0
4,2018-01,National,PARIS MONTPARNASSE,SAINT-PIERRE-DES-CORPS,61,391,2,NaN,61,7.796995,...,6,0,21.212121,42.424242,9.090909,21.212121,6.060606,0.000000,157.0,4.0


In [37]:
df = pd.merge(df, geo, how = 'left', left_on='gare_depart',right_on='Intitulé gare')
df.rename(columns={'Longitude':'Longitude_gare_depart','Latitude':'Lattitude_gare_depart'}, inplace=True)
df = pd.merge(df, geo, how = 'left', left_on='gare_arrivee',right_on='Intitulé gare')
df.rename(columns={'Longitude':'Longitude_gare_arrivee','Latitude':'Lattitude_gare_arrivee'}, inplace=True)
df.drop(columns=['Intitulé gare_x', 'Intitulé gare_y'], inplace=True)

In [38]:
df.head()

,date,service,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,commentaire_annulation,nb_train_depart_retard,retard_moyen_depart,...,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs,nombre_objets_perdus_depart,nombre_objets_perdus_arrivee,Longitude_gare_depart,Lattitude_gare_depart,Longitude_gare_arrivee,Lattitude_gare_arrivee
0,2018-01,National,BORDEAUX SAINT-JEAN,PARIS MONTPARNASSE,141,870,5,NaN,289,11.247809,...,10.924370,15.966387,5.042017,0.840336,105.0,157.0,-0.556697,44.825873,2.320514,48.841172
1,2018-01,National,BORDEAUX SAINT-JEAN,PARIS MONTPARNASSE,141,870,5,NaN,289,11.247809,...,10.924370,15.966387,5.042017,0.840336,105.0,157.0,-0.556697,44.825873,2.320514,48.841172
2,2018-01,National,LA ROCHELLE,PARIS MONTPARNASSE,165,222,0,NaN,8,2.875000,...,38.461538,11.538462,3.846154,0.000000,22.0,157.0,-1.145305,46.152690,2.320514,48.841172
3,2018-01,National,LA ROCHELLE,PARIS MONTPARNASSE,165,222,0,NaN,8,2.875000,...,38.461538,11.538462,3.846154,0.000000,22.0,157.0,-1.145305,46.152690,2.320514,48.841172
4,2018-01,National,PARIS MONTPARNASSE,QUIMPER,220,248,1,NaN,37,9.501351,...,15.384615,19.230769,0.000000,0.000000,157.0,26.0,2.320514,48.841172,-4.092103,47.994455


In [39]:
df[df['Longitude_gare_depart'].isna()]['gare_depart'].unique()


array(['BARCELONA', 'GENEVE', 'MADRID', 'LAUSANNE', 'FRANCFORT', 'ITALIE',
       'ZURICH', 'VALENCE TGV RHÔNES-ALPES SUD', 'STUTTGART'],
      dtype=object)

In [40]:
df[df['Longitude_gare_arrivee'].isna()]['gare_arrivee'].unique()

array(['FRANCFORT', 'ZURICH', 'VALENCE TGV RHÔNES-ALPES SUD', 'STUTTGART',
       'BARCELONA', 'ITALIE', 'LAUSANNE', 'MADRID', 'GENEVE'],
      dtype=object)

In [42]:
# Save data
df.to_csv('./regularite-mensuelle-tgv-ext-data.csv')